In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import time
import torch
model = YOLO("E:\AbioticNeuron\DS\DS Projects\Vehicle Tracing\Deepsort\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\yolov8l.pt")

<>:6: SyntaxWarning: invalid escape sequence '\A'
<>:6: SyntaxWarning: invalid escape sequence '\A'
C:\Users\pradh\AppData\Local\Temp\ipykernel_18920\3001483839.py:6: SyntaxWarning: invalid escape sequence '\A'
  model = YOLO("E:\AbioticNeuron\DS\DS Projects\Vehicle Tracing\Deepsort\Tracking-and-counting-Using-YOLOv8-and-DeepSORT\yolov8l.pt")


# DeepSORT

In [2]:

from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

In [3]:
# Define the video path
video_path = # file path

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
frames = []

unique_track_ids = set()

In [5]:

# Open video capture
cap = cv2.VideoCapture(video_path)
out = cv2.VideoWriter("output_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (int(cap.get(3)), int(cap.get(4))))

# Tracking variables
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()
unique_track_ids = set()
frames = []
print(cap.isOpened())

while cap.isOpened():
    ret, frame = cap.read()
    print(ret)
    if not ret:
        break
    
    og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = og_frame.copy()

    # Run YOLO model with car class (class 2 in COCO dataset)
    results = model(frame, classes=[2])
    
    for result in results:
        boxes = result.boxes  # Bounding boxes
        cls = boxes.cls.tolist()  # Class IDs
        xyxy = boxes.xyxy
        conf = boxes.conf
        xywh = boxes.xywh.cpu().numpy()
        
        # Convert to NumPy
        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        
        # Update tracker with car detections
        tracks = tracker.update(xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            x1, y1, x2, y2 = track.to_tlbr()
            w, h = x2 - x1, y2 - y1

            # Draw bounding box
            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(og_frame, f"Car-{track_id}", (int(x1), int(y1) - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            unique_track_ids.add(track_id)
        
    # Update FPS
    current_time = time.perf_counter()
    elapsed = (current_time - start_time)
    counter += 1
    if elapsed > 1:
        fps = counter / elapsed
        counter = 0
        start_time = current_time

    # Draw car count
    car_count = len(unique_track_ids)
    cv2.putText(og_frame, f"Car Count: {car_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Write output
    out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

cap.release()
out.release()
cv2.destroyAllWindows()


True
True

0: 384x640 14 cars, 718.1ms
Speed: 5.5ms preprocess, 718.1ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 13 cars, 707.0ms
Speed: 3.9ms preprocess, 707.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 13 cars, 661.9ms
Speed: 4.3ms preprocess, 661.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 14 cars, 776.1ms
Speed: 3.1ms preprocess, 776.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 13 cars, 703.7ms
Speed: 4.3ms preprocess, 703.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 14 cars, 682.8ms
Speed: 3.5ms preprocess, 682.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 13 cars, 628.6ms
Speed: 3.2ms preprocess, 628.6ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 13 cars, 635.7ms
Speed: 2.8ms preprocess, 635.7ms infe